# Getting Started with OPERA DSWx Product
---

This notebook serves as an introduction to the [OPERA Dynamic Water eXtent (DSWx) product]((https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_DSWX_URS309746.pdf)) and visualizing relevant raster layers for reservoir monitoring applications

In [ ]:
# Notebook dependencies
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import rioxarray as rio

import hvplot.xarray
from bokeh.models import FixedTicker
import geoviews as gv
gv.extension('bokeh')



### Metadata
---

HLS products provide surface reflectance (SR) data from the Operational Land Imager (OLI) aboard the Landsat-8 remote sensing satellite and the Multi-Spectral Instrument (MSI) aboard the Sentinel-2 A/B remote sensing satellite. HLS products are distributed over projected map coordinates aligned with the Military Grid Reference System (MGRS). Each tile covers 109.8 kilometers squared divided into 3660 rows and 3660 columns at 30 meter pixel spacing. Each tile overlaps neighbors by 4900 meters in each direction.



### Raster Layers
___

The **DSWx** product is distributed as a set of Cloud-Optimized GeoTIFF (COG) files to enable download of only particular layers of interest to a given user. All L3 DSWx layers are stored in files following GeoTIFF format specifications.

___

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DSWx-HLS_T11SQA_20230828T181921Z_20230831T000636Z_S2A_30_v1.0_B01_WTR.tif'

data = rio.open_rasterio(LOCAL_PATH)
crs = data.rio.crs
data = data.rename({'x':'easting', 'y':'northing', 'band':'band'}).squeeze()

## **Band 1: Water classification (WTR)**
***


**Data Type:**  UInt8 <br>
**Description:** Masked interpreted water classification layer. This represents pixel-wise classification into one of three water
classes (not water, open water, and partial surface water), cloud/cloud shadow class, or no data classes.


**Layer Values:**<br> 
* **0:** Not Water – an area with valid reflectance data that is not open water (class 1), partial surface water (class 2), or
cloud/cloud shadow (class 9). Masking can result in “not water” (class 0) where land cover masking is applied<br>
* **1:** Open Water – an area that is entirely water and unobstructed to the sensor, including obstructions by vegetation, terrain,
and buildings <br>
* **2:** Partial Surface Water – an area that is at least 50% and less than 100% open water. This may be referred to as “subpixel
inundation” when referring to a pixel’s area. Examples include inundated sinkholes, floating vegetation, and pixels bisected by
coastlines <br> 
* **8:** Snow/Ice - an area identified as snow/ice according to input HLS Fmask quality assurance (QA) data <br>
* **9:** Cloud/Cloud Shadow – an area identified as cloud, cloud shadow, or snow/ice according to input quality assurance (QA)
data <br>
* **255:** Fill value (no data)  <br> 

In [ ]:
# Defines colormap for visualization
levels = [0, 0.9, 1.9, 2.9, 7.9, 8.9, 10]
color_key = {
    "Not Water": "#ffffff",
    "Open Water": "#0000ff",
    "Partial Surface Water": "#00ff00",
    "Reserved": "#000000",
    "Snow/Ice": "#00ffff",
    "Clouds/Cloud Shadow": "#7f7f7f"
}

ticks = [0.5, 1.5, 2.5, 5.5, 8.5, 9.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

In [ ]:
# Creates basemap
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000, padding=0.1)
dswx = data.where(data>0)

In [ ]:
dswx.hvplot.image(x='easting', 
                  y='northing', 
                  rasterize=True, 
                  dynamic=True, 
                  aspect='equal', 
                  frame_width=500, 
                  frame_height=500,  
                  clim=(0,10),
                  alpha=0.8,
                  crs=crs).opts(title=f"B01_WTR",color_levels=levels,cmap=tuple(color_key.values()),
                                    colorbar_opts={'ticker':ticker,'major_label_overrides':labels}) * base

## **Band 2: Binary water (BWTR)**
***


**Data Type:**  UInt8 <br>
**Description:** The binary water map is derived from the WTR layer as a union of water classes (open water and partial surface
water) into a binary map indicating areas with and without water. This layer is meant to provide users with a quick view for
water/no-water. Invalid data classes (cloud/cloud shadow and fill value) are also provided to indicate areas in which the binary
classification does not provide water/no-water classification.


**Layer Values:**<br> 
* **0:** Not water – an area with valid reflectance data that is not water (class 1) and not cloud/cloud shadow (class 9)<br>
* **1:** Water – an area classified as “open water” or “partial surface water” (see WTR layer) <br>
* **8:** Snow/Ice - an area identified as snow/ice according to input HLS Fmask quality assurance (QA) data <br>
* **9:** Cloud/Cloud Shadow – an area identified as cloud, cloud shadow, or snow/ice according to input QA data <br>
* **255:** Fill value (no data)  <br> 

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DSWx-HLS_T11SQA_20230828T181921Z_20230831T000636Z_S2A_30_v1.0_B02_BWTR.tif'
data = rio.open_rasterio(LOCAL_PATH)
crs = data.rio.crs
data = data.rename({'x':'easting', 'y':'northing', 'band':'band'}).squeeze()
bwtr = data.where((data!=255) & (data!=0))

In [ ]:
# Defines colormap for visualization
levels = [0, 0.9, 1.9, 7.9, 8.9, 10]
color_key = {
    "Not Water": "#ffffff",
    "Water": "#0000ff",
    "Reserved": "#000000",
    "Snow/Ice": "#00ffff",
    "Clouds/Cloud Shadow": "#7f7f7f"
}

ticks = [0.5, 1.5, 5.5, 8.5, 9.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

bwtr.hvplot.image(x='easting', 
                  y='northing',
                  rasterize=True, 
                  dynamic=True, 
                  aspect='equal', 
                  frame_width=500, 
                  frame_height=500,
                  clim=(0,10),
                  alpha=0.8,
                  crs=crs).opts(title=f"B02_BWTR",xlabel='Longitude', 
                                    ylabel='Latitude',color_levels=levels,cmap=tuple(color_key.values()),
                                    colorbar_opts={'ticker':ticker,'major_label_overrides':labels}) * base

## **Band 5: Interpretation of diagnostic layer into water classes (WTR-1)**
***


**Data Type:**  UInt8 <br>
**Description:** Classification of DIAG layer results into open water, partial surface water, and no-water.


**Layer Values:**<br> 
* **0:** Not Water - an area with valid reflectance data that is not open water (class 1) or partial surface water (class 2)<br>
* **1:** Open Water – an area that is entirely water and unobstructed to the sensor, including obstructions by vegetation, terrain,
and buildings <br>
* **2:** Partial Surface Water – an area that is at least 50% and less than 100% open water. This may be referred to as “subpixel
inundation” when referring to a pixel’s area. Examples include wetlands, water bodies with floating vegetation, and pixels
bisected by coastlines <br> 
* **255:** Fill value (no data)  <br> 

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DSWx-HLS_T11SQA_20230828T181921Z_20230831T000636Z_S2A_30_v1.0_B05_WTR-1.tif'

data = rio.open_rasterio(LOCAL_PATH)
crs = data.rio.crs
data = data.rename({'x':'easting', 'y':'northing', 'band':'band'}).squeeze()
wtr1 = data.where((data!=255) & (data!=0))

In [ ]:
# Defines colormap for visualization
levels = [0, 0.6, 1.3, 2]
color_key = {
    "Not Water": "#ffffff",
    "Open Water": "#0000ff",
    "Partial Surface Water": "#00ff00",
}

ticks = [0.25, 0.9, 1.6]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

wtr1.hvplot.image(x='easting', 
                  y='northing', 
                  rasterize=True, 
                  dynamic=True, 
                  aspect='equal', 
                  frame_width=500, 
                  frame_height=500,  
                  clim=(0,2),
                  alpha=0.8,
                  crs=crs).opts(title=f"B05_WTR-1",xlabel='Longitude', ylabel='Latitude',
                                color_levels=levels,cmap=tuple(color_key.values()),
                                colorbar_opts={'ticker':ticker,'major_label_overrides':labels}) * base

## **Band 6: Interpreted layer refined using land cover and terrain shadow testing (WTR-2)**
***


**Data Type:**  UInt8 <br>
**Description:** The WTR-2 layer is derived from the WTR-1 (Layer 5) outcome by applying additional tests based on land cover
and terrain shadow information to mask (eliminate) false-positive water detections.

**Layer Values:**<br> 
* **0:** Not Water - an area with valid reflectance data that is not open water (class 1) or partial surface water (class 2)<br>
* **1:** Open Water – an area that is entirely water and unobstructed to the sensor, including obstructions by vegetation, terrain,
and buildings <br>
* **2:** Partial Surface Water – an area that is at least 50% and less than 100% open water. This may be referred to as “subpixel
inundation” when referring to a pixel’s area. Examples include wetlands, water bodies with floating vegetation, and pixels
bisected by coastlines <br> 
* **255:** Fill value (no data)  <br> 

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DSWx-HLS_T11SQA_20230828T181921Z_20230831T000636Z_S2A_30_v1.0_B06_WTR-2.tif'

data = rio.open_rasterio(LOCAL_PATH)
crs = data.rio.crs
data = data.rename({'x':'easting', 'y':'northing', 'band':'band'}).squeeze()
wtr2 = data.where((data!=255) & (data!=0))

In [ ]:
# Defines colormap for visualization
levels = [0, 0.6, 1.3, 2]
color_key = {
    "Not Water": "#ffffff",
    "Open Water": "#0000ff",
    "Partial Surface Water": "#00ff00",
}

ticks = [0.25, 0.9, 1.6]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

wtr2.hvplot.image(x='easting', 
                  y='northing', 
                  rasterize=True, 
                  dynamic=True, 
                  aspect='equal', 
                  frame_width=500, 
                  frame_height=500,  
                  clim=(0,2),
                  alpha=0.8,
                  crs=crs).opts(title=f"B06_WTR-2",xlabel='Longitude', 
                                    ylabel='Latitude',color_levels=levels,cmap=tuple(color_key.values()),
                                    colorbar_opts={'ticker':ticker,'major_label_overrides':labels}) * base